In [4]:
#imports
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

In [10]:
#preproccessing

In [9]:
trainset=None
testset=None

trainloader=None
testloader=None
#...

In [7]:
#Model
class MyLSTM(nn.Module):
    def __init__(self,n_inputs,n_hidden=256,n_layers=3):
        super().__init__()
        self.n_hidden=n_hidden
        
        self.lstm=nn.LSTM(
            input_size=n_inputs,
            hidden_size=n_hidden,
            num_layers=n_layers,
            dropout=0.2,
            batch_first=True
        )
        
        self.fc1=nn.Linear(self.n_hidden,1)
        
    def forward(self,x):
        _,(h_t,_)=self.lstm(x)
        
        x=self.fc1(h_t[-1])
        return x

In [8]:
#Init Model,criterion,optimizer
n_inputs=0  ####
model=MyLSTM(n_inputs)
criterion=nn.MSELoss
optimizer=optim.Adam(model.parameters(),lr=0.0001)
training_losses,validation_losses=[],[]

In [6]:
#Model Training

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
epochs= 10
steps,running_loss,test_loss=0,0,0
model.to(device)
min_validation_loss=np.Infinity
validate_every = 50

for e in range(epochs):
    
    #training
    for inputs, labels in trainloader:###
        steps+=1
        inputs, labels= inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logps = model.forward(inputs)
        loss = criterion(logps, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        #testing
        if steps % validate_every == 0:
            validation_loss = 0
            model.eval()
            with torch.no_grad():
                for inputs, labels in testloader:
                    inputs, labels= inputs.to(device), labels.to(device)
                    logps = model.forward(inputs)
                    v_loss = criterion(logps, labels)
                    validation_loss+=v_loss.item()
            
            model.train()
            
            avg_validation_loss=validation_loss/len(testloader)
            
            print(f"Epoch {e+1}/{epochs},  "
                  f"Training loss: {running_loss/validate_every:.3f},  "
                  f"Validation loss: {avg_validation_loss:.3f},  ")
            
            training_losses.append(running_loss/validate_every)
            validation_losses.append(avg_validation_loss)
            
            #Saving better model
            if avg_validation_loss<min_validation_loss:
                print(f"Saving better model")
                min_validation_loss=avg_validation_loss
                torch.save( model.state_dict(), 'lstm_model_dict.pt')
              
            running_loss = 0


In [ ]:
#plot
%matplotlib inline
%config inlineBackend.figure_format='retina'

import matplotlib.pyplot as plt
plt.plot(training_losses,label='Train loss')
plt.plot(validation_losses,label='Validation loss')
plt.legend()
plt.show()

In [ ]:
#prediction
#...

In [ ]:
#Saving Model
checkpoint={'state_dict': model.state_dict(),
            'inputs': n_inputs,
            'optimizer': optimizer,
            'criterion': criterion,
            'model': model}
torch.save(checkpoint, 'checkpoint.pth')